In [ ]:
from my_code.datasets.surreal_dataset_3dc import TemplateSurrealDataset3DC

num_evecs = 32
# create the dataset
dataset = TemplateSurrealDataset3DC(
    shape_path=f'/home/s94zalek_hpc/3D-CODED/data/datas_surreal_train.pth',
    num_evecs=32,
    use_cuda=False,
    cache_lb_dir=None,
    return_evecs=True
)    

In [ ]:
from my_code.sign_canonicalization.training import predict_sign_change
import networks.diffusion_network as diffusion_network
import my_code.utils.plotting_utils as plotting_utils
import matplotlib.pyplot as plt
    

In [ ]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
net_path = '/home/s94zalek_hpc/shape_matching/my_code/experiments/sign_estimator_no_aug/40000.pth'
num_evecs = 32
evecs_per_support = 4
input_type = 'wks'


net = diffusion_network.DiffusionNet(
    in_channels=num_evecs,
    out_channels=num_evecs // evecs_per_support,
    cache_dir=None,
    input_type=input_type,
    k_eig=128,
    n_block=6
    ).to(device)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
net.load_state_dict(torch.load(net_path, map_location=device))


In [ ]:
from tqdm import tqdm
import utils.geometry_util as geometry_util
import robust_laplacian
import scipy.sparse.linalg as sla
import utils.geometry_util as geometry_util

tqdm._instances.clear()

# shapes_to_test = test_shapes
# net.cache_dir = test_diff_folder

shapes_to_test = [dataset[0]['first']]
net.cache_dir = '/home/s94zalek_hpc/shape_matching/notebooks/24.07.2024'
         
             
              
iterator = tqdm(range(1000))
incorrect_signs_list = torch.tensor([])
curr_iter = 0

for epoch in range(len(iterator) // len(shapes_to_test)):
    
    # train_shapes_shuffled = train_shapes.copy()
    # np.random.shuffle(test_shapes_list)
    
    
    for curr_idx in range(len(shapes_to_test)):     


        ##############################################
        # Select a shape
        ##############################################
        
        test_shape = shapes_to_test[curr_idx]    
        
        verts = test_shape['verts'].unsqueeze(0).to(device)
        faces = test_shape['faces'].unsqueeze(0).to(device)
        evecs_orig = test_shape['evecs'][:, :num_evecs].unsqueeze(0).to(device)

        ##############################################
        # Set the signs on shape 0
        ##############################################

        # create a random combilation of +1 and -1, length = feature_dim
        sign_gt_0 = torch.randint(0, 2, (num_evecs,)).float().to(device)
        
        sign_gt_0[sign_gt_0 == 0] = -1
        sign_gt_0 = sign_gt_0.float().unsqueeze(0)

        # multiply evecs [6890 x 16] by sign_flip [16]
        evecs_flip_0 = evecs_orig * sign_gt_0
        
        # predict the sign change
        with torch.no_grad():
            sign_pred_0, supp_vec_0, _ = predict_sign_change(
                net, verts, faces, evecs_flip_0, evecs_cond=None, input_type=input_type)
        
        ##############################################
        # Set the signs on shape 1
        ##############################################
        
        # create a random combilation of +1 and -1, length = feature_dim
        sign_gt_1 = torch.randint(0, 2, (num_evecs,)).float().to(device)
        
        sign_gt_1[sign_gt_1 == 0] = -1
        sign_gt_1 = sign_gt_1.float().unsqueeze(0)
        
        # multiply evecs [6890 x 16] by sign_flip [16]
        evecs_flip_1 = evecs_orig * sign_gt_1
        
        # predict the sign change
        with torch.no_grad():
            sign_pred_1, supp_vec_1, _ = predict_sign_change(
                net, verts, faces, evecs_flip_1, evecs_cond=None, input_type=input_type)
        
        ##############################################
        # Calculate the loss
        ##############################################
        
        # calculate the ground truth sign difference
        sign_diff_gt = sign_gt_1 * sign_gt_0
        
        # calculate the sign difference between predicted evecs
        sign_diff_pred = sign_pred_1 * sign_pred_0
        
        sign_correct = sign_diff_pred.sign() * sign_diff_gt.sign() 
        
        
        # count the number of incorrect signs
        count_incorrect_signs = (sign_correct < 0).int().sum()
            
        # incorrect_signs_list.append(count_incorrect_signs)
        incorrect_signs_list = torch.cat([incorrect_signs_list, torch.tensor([count_incorrect_signs])])
        
        
        iterator.set_description(f'Mean incorrect signs {incorrect_signs_list.float().mean():.2f} / {num_evecs}, max {incorrect_signs_list.max()}')
        iterator.update(1)
        # if count_incorrect_signs > 7:
        #     raise ValueError('Too many incorrect signs')
    
    
print(f'Results for {len(incorrect_signs_list)} test shapes')
print(f'Incorrect signs per shape: {incorrect_signs_list.float().mean():.2f} / {num_evecs}')

print('Max incorrect signs', incorrect_signs_list.max())

print()
# print('Shape idx', curr_idx)
print('GT', sign_diff_gt)
print('PRED', sign_diff_pred)
print('Correct', sign_correct)
print(f'Incorrect signs {torch.sum(sign_correct != 1)} / {num_evecs}')
print(incorrect_signs_list)


In [ ]:
import numpy as np

# read /home/s94zalek_hpc/shape_matching/data/SURREAL_full/full_datasets/dataset_3dc_corrected_noAug_32/train/C_gt_xy.txt
data_Cxy = np.loadtxt('/home/s94zalek_hpc/shape_matching/data/SURREAL_full/full_datasets/dataset_3dc_corrected_noAug_32/train/C_gt_xy_0_11500.txt')
data_Cxy = torch.from_numpy(data_Cxy).reshape(data_Cxy.shape[0], 32, 32)

data_evals = np.loadtxt('/home/s94zalek_hpc/shape_matching/data/SURREAL_full/full_datasets/dataset_3dc_corrected_noAug_32/train/evals_0_11500.txt')
data_evals = torch.from_numpy(data_evals)

In [ ]:
data_Cxy_unproc = np.loadtxt('/home/s94zalek_hpc/shape_matching/data/SURREAL_full/full_datasets/dataset_3dc_faceNorm_32/train/C_gt_xy_0_11500.txt')
data_Cxy_unproc = torch.from_numpy(data_Cxy_unproc).reshape(data_Cxy_unproc.shape[0], 32, 32)

In [ ]:
l = 0
h = 32

# get 4 random indices
idxs = np.random.randint(0, data_Cxy.shape[0], 4)

fig, axs = plt.subplots(2, 4, figsize=(15, 8))

plotting_utils.plot_Cxy(fig, axs[0, 0], data_Cxy[idxs[0]],
                        f'Proc {idxs[0]}', l, h, show_grid=False, show_colorbar=False)
plotting_utils.plot_Cxy(fig, axs[0, 1], data_Cxy[idxs[1]],
                        f'{idxs[1]}', l, h, show_grid=False, show_colorbar=False)
plotting_utils.plot_Cxy(fig, axs[0, 2], data_Cxy[idxs[2]],
                        f'{idxs[2]}', l, h, show_grid=False, show_colorbar=False)
plotting_utils.plot_Cxy(fig, axs[0, 3], data_Cxy[idxs[3]],
                        f'{idxs[3]}', l, h, show_grid=False, show_colorbar=False)

plotting_utils.plot_Cxy(fig, axs[1, 0], data_Cxy_unproc[idxs[0]],
                        f'Unproc {idxs[0]}', l, h, show_grid=False, show_colorbar=False)
plotting_utils.plot_Cxy(fig, axs[1, 1], data_Cxy_unproc[idxs[1]],
                        f'{idxs[1]}', l, h, show_grid=False, show_colorbar=False)
plotting_utils.plot_Cxy(fig, axs[1, 2], data_Cxy_unproc[idxs[2]],
                        f'{idxs[2]}', l, h, show_grid=False, show_colorbar=False)
plotting_utils.plot_Cxy(fig, axs[1, 3], data_Cxy_unproc[idxs[3]],
                        f'{idxs[3]}', l, h, show_grid=False, show_colorbar=False)

plt.show()

In [ ]:
# apply standard scaling and PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import pandas as pd


scaler = StandardScaler()
pca = PCA(n_components=32)

C_xy_pred_list_scaled = scaler.fit_transform(data_Cxy.reshape(data_Cxy.shape[0], -1))
C_xy_pred_list_pca = pca.fit_transform(C_xy_pred_list_scaled)


fig, axs = plt.subplots(1, 1, figsize=(5, 5))

# plot explained variance
axs.plot(pca.explained_variance_ratio_, '.-')
axs.set_title('Explained variance ratio')


pca_df = pd.DataFrame(C_xy_pred_list_pca[::100, :2], columns=[f'PCA_{i}' for i in range(2)])
# only keep 1000 elements, evenly distributed

# pca_df = pd.DataFrame(C_xy_pred_list_pca, columns=[f'PCA_{i}' for i in range(32)])

# pca_df['name'] = names_y
# pca_df['body_type'] = [i // 10 for i in range(C_xy_pred_list_pca.shape[0])]
# pca_df['pose'] = [i % 10 for i in range(C_xy_pred_list_pca.shape[0])]


# use numbers as markers
sns.pairplot(pca_df, diag_kind='kde') #, hue='body_type', palette='tab10')

print('PCA on predicted')
plt.show()



In [ ]:
# apply standard scaling and PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import pandas as pd


scaler = StandardScaler()
pca = PCA(n_components=32)

C_xy_pred_list_scaled = scaler.fit_transform(data_Cxy_unproc.reshape(data_Cxy_unproc.shape[0], -1))
C_xy_pred_list_pca = pca.fit_transform(C_xy_pred_list_scaled)


fig, axs = plt.subplots(1, 1, figsize=(5, 5))

# plot explained variance
axs.plot(pca.explained_variance_ratio_, '.-')
axs.set_title('Explained variance ratio')


pca_df = pd.DataFrame(C_xy_pred_list_pca[::100, :2], columns=[f'PCA_{i}' for i in range(2)])
# only keep 1000 elements, evenly distributed

# pca_df = pd.DataFrame(C_xy_pred_list_pca, columns=[f'PCA_{i}' for i in range(32)])

# pca_df['name'] = names_y
# pca_df['body_type'] = [i // 10 for i in range(C_xy_pred_list_pca.shape[0])]
# pca_df['pose'] = [i % 10 for i in range(C_xy_pred_list_pca.shape[0])]


# use numbers as markers
sns.pairplot(pca_df, diag_kind='kde') #, hue='body_type', palette='tab10')

print('PCA on predicted')
plt.show()



In [ ]:
tensor_1 = torch.tensor([1,2,3,4,5])
tensor_2 = torch.tensor([2])

# torch.cat([tensor_1.sum(), tensor_2])
# tensor_1.sum()
torch.cat([
    torch.tensor([tensor_1.sum()]),
    tensor_2])